In [2]:
import imageio.v3 as iio
import requests
import numpy as np
import os
from datetime import datetime
from dotenv import load_dotenv
import sys  
import colorsys
import csv

load_dotenv()

True

In [7]:
# write csv header
fields = ['calc_score', 'humidity', 'cloud_cover', 'visibility', 'user_score', 'lat', 'long', 'date_time', 'image_url']

with open('./data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    row = ['a', 'b', 'c']
    writer.writerow(fields)

In [10]:
# Make api call to flickr
# for some reason, flickr key isn't found (KeyError)
FLICKR_API_KEY = os.getenv("FLICKR_API_KEY")

flickr_api_url = 'https://www.flickr.com/services/rest/?method=flickr.photos.search' +\
            '&api_key=' + FLICKR_API_KEY +\
            '&tags=sunset' +\
            '&text=sunset%20landscape' +\
            '&content_type=1' +\
            '&has_geo=1' +\
            '&min_upload_date=2000-01-01' +\
            '&sort=date-posted-asc' +\
            '&extras=date_taken%2C+date_upload%2C+geo%2C+url_o' +\
            '&per_page=250' +\
            '&page=6' +\
            '&format=json&nojsoncallback=1'
            # '&is_getty=1' +\

flickr_res = requests.get(flickr_api_url)

if(flickr_res.status_code < 200 or flickr_res.status_code >= 300): 
    print('Unsuccessful request. Error code: {}'.format(flickr_res.status_code))
    sys.exit()
else:
    print("Flickr API call was successful.")
    flickr_res = flickr_res.json()

print('Page:')
print(flickr_res['photos']['page'])
print('Total number of photos:')
print(flickr_res['photos']['total'])
print("Length of Flickr API response: ", len(flickr_res['photos']['photo']))


Flickr API call was successful.
Page:
6
Total number of photos:
260888
Length of Flickr API response:  249


In [11]:
# Process multiple images at once
VISUALCROSSING_API_KEY = os.environ["VISUALCROSSING_API_KEY"]
DATE_TIME_FORMAT_STRING = "%Y-%m-%d %H:%M:%S"
SAMPLE_SCALE = 5

def score_sunset(image):
    sunset_pixel_counter = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rVal = image[i,j,0]
            gVal = image[i,j,1]  
            bVal = image[i,j,2]
            hVal = colorsys.rgb_to_hls(rVal, gVal, bVal)[0]*360
            if(hVal <= 45): # oranges are weighted less
                sunset_pixel_counter += 0.7
            elif(hVal >= 270): # reds and magentas are weighted fully
                sunset_pixel_counter +=1

    total_image_pixels = image.shape[1] *  image.shape[0]
    sunset_score = ((sunset_pixel_counter / total_image_pixels) * 10)
    sunset_score = round(sunset_score, 3)
    return sunset_score

for i in range(len(flickr_res['photos']['photo'])):
    # Get and load pic from flickr res
    photo = flickr_res['photos']['photo'][i]
    imgUrl = f'https://live.staticflickr.com/{photo["server"]}/{photo["id"]}_{photo["secret"]}_b.jpg'
    pic = iio.imread(imgUrl)
    if len(pic.shape) != 3: # skip over grayscale images
        continue
    
    # GET WEATHER DATA FOR IMAGE
    # nearest hour of sunset for picture
    date_taken = datetime.strptime(photo["datetaken"], DATE_TIME_FORMAT_STRING)
    nearest_sunset_hour = date_taken.hour
    if date_taken.minute > 30:
        nearest_sunset_hour +=  1
    
    # make api call for picture
    visualcrossing_history_api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?" +\
                            "&aggregateHours=1" +\
                            "&startDateTime=" + str(date_taken.date()) +\
                            "&endDateTime=" + str(date_taken.date())  +\
                            "&unitGroup=metric" +\
                            "&contentType=json" +\
                            "&dayStartTime=" + str(nearest_sunset_hour) + ":00:00" +\
                            "&dayEndTime=" + str(nearest_sunset_hour) + ":01:00" +\
                            "&location=" + photo["latitude"] + "%2C%20" + photo["longitude"] +\
                            "&key=" + VISUALCROSSING_API_KEY
    weather_res = requests.get(visualcrossing_history_api_url)

    if(weather_res.status_code != 200): 
        print('Unsuccessful request. Error code: {}'.format(weather_res.status_code))
        sys.exit()
    else:
        # print('Visual Crossing API call was successful for index ', i)
        weather_res = weather_res.json()
    
    # get weather data of picture
    try:
        nearest_hour_data = weather_res['locations'][photo["latitude"] + ", " + photo["longitude"]]['values'][0]
        humidity = nearest_hour_data['humidity']
        cloud_cover = nearest_hour_data['cloudcover']
        visibility = nearest_hour_data['visibility']
    except KeyError:
        if(weather_res['errorCode'] == -994): # api call quota hit
            print(weather_res)
            break
        else:
            # handles errorCode 999 (weird stuff)
            print(weather_res)
            print(humidity, cloud_cover, visibility)
        
    # skip images with empty weather data
    if(humidity == None or cloud_cover == None or visibility == None):
        print("image" , i, "doesn't have all data:")
        print(humidity, cloud_cover, visibility)
        continue

    # ANALYZE PHOTO
    length = pic.shape[0]
    width = pic.shape[1]
    sys_sample_pic = pic[0:length:SAMPLE_SCALE, 0:width:SAMPLE_SCALE, :]
    calc_score = score_sunset(sys_sample_pic)

    # WRITE TO CSV
    row = [calc_score, humidity, cloud_cover, visibility, "", photo["latitude"], photo["longitude"], date_taken, imgUrl]
    f = open('./data.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerow(row)
    f.close()
    print("Image",  i, ", ", imgUrl, ", has been written to csv.")

Image 0 ,  https://live.staticflickr.com/737/22725212332_25ff95ae8d_b.jpg , has been written to csv.
Image 1 ,  https://live.staticflickr.com/4600/38080825235_0a93c7ef1a_b.jpg , has been written to csv.
Image 2 ,  https://live.staticflickr.com/1633/25649054963_0c0058b052_b.jpg , has been written to csv.
Image 3 ,  https://live.staticflickr.com/1305/564414484_67e28b824f_b.jpg , has been written to csv.
Image 4 ,  https://live.staticflickr.com/605/22905234193_685d812265_b.jpg , has been written to csv.
Image 5 ,  https://live.staticflickr.com/5480/9658710469_bca84614b4_b.jpg , has been written to csv.
Image 6 ,  https://live.staticflickr.com/65535/52082888855_f07de1cdea_b.jpg , has been written to csv.
Image 7 ,  https://live.staticflickr.com/1554/25729517591_e2925714f1_b.jpg , has been written to csv.
Image 8 ,  https://live.staticflickr.com/6059/6296210594_1856b70fcf_b.jpg , has been written to csv.
Image 9 ,  https://live.staticflickr.com/65535/50252472143_eba9c08b72_b.jpg , has been 

In [21]:
test = flickr_res['photos']['photo'][27]
imgUrl = f'https://live.staticflickr.com/{test["server"]}/{test["id"]}_{test["secret"]}_b.jpg'
sample = iio.imread(imgUrl)
len(sample.shape)

2